## Modelo de lenguaje con tokenización por palabras

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


In [ ]:
pip install numpy pandas scikit-learn tensorflow keras

**Importamos las librerias necesarias**

In [18]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.callbacks import Callback

**Descargamos el dataset 20 News Groups**

In [3]:
newsgroups = fetch_20newsgroups(subset='train', categories=None)
texts = newsgroups.data

**Imprimimos las categorías cargadas**

In [16]:
print(newsgroups.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


**Procesamos los datos de texto**

In [5]:
def preprocess_text(text):
    text = text.lower()  # Convertimos a minúsculas
    text = re.sub(r'\d+', '', text)  # Eliminamos números
    text = re.sub(r'\s+', ' ', text)  # Eliminamos espacios en blanco repetidos
    return text

texts = [preprocess_text(text) for text in texts]

**Tokenización**

In [6]:
tokenizer = Tokenizer(num_words=20000)  # Limitamos a las 20,000 palabras más frecuentes
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

**Longitud máxima de secuencias**

In [7]:
max_sequence_length = 100

**Padding**

In [8]:
# Padding de las secuencias para que todas tengan la misma longitud
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

In [9]:
# Creamos la variable objetivo y la entrada (usaremos la última palabra de cada secuencia como objetivo)
X_sequences = np.array([sequence[:-1] for sequence in X_padded if len(sequence) > 1])  # Todo menos la última palabra
y_sequences = np.array([sequence[-1] for sequence in X_padded if len(sequence) > 1])  # La última palabra como objetivo

**Dividimos el dataset en entrenamiento y validación**

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

**Ahora definimos el modelo**

**Parámetros**

In [11]:
# Parámetros del modelo
embedding_dim = 50
hidden_units = 64
vocab_size = len(tokenizer.word_index) + 1  # El tamaño del vocabulario

In [12]:
# Callback para calcular la perplejidad al final de cada época
class PerplexityCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        if val_loss is not None:
            perplexity = np.exp(val_loss)
            print(f'Epoch {epoch + 1} - Perplexity: {perplexity:.2f}')

In [13]:
# Creamos el modelo LSTM
def create_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length - 1))
    model.add(LSTM(hidden_units, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(optimizer=RMSprop(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Creamos el modelo GRU
def create_gru_model():
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length - 1))
    model.add(GRU(hidden_units, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(optimizer=RMSprop(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Creamos el modelo SimpleRNN
def create_rnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length - 1))
    model.add(SimpleRNN(hidden_units, dropout=0.2))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(optimizer=RMSprop(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

**Entrenamos el modelo**

In [14]:
# Elegimos el modelo a entrenar (puedes cambiar entre create_lstm_model, create_gru_model o create_rnn_model)
model = create_lstm_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
# Entrenamos el modelo
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, callbacks=[PerplexityCallback()])

Epoch 1/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.0682 - loss: 10.6919Epoch 1 - Perplexity: 6490.71
283/283 ━━━━━━━━━━━━━━━━━━━━ 64s 199ms/step - accuracy: 0.0682 - loss: 10.6892 - val_accuracy: 0.0844 - val_loss: 8.7781
Epoch 2/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.0752 - loss: 8.6474Epoch 2 - Perplexity: 3868.34
283/283 ━━━━━━━━━━━━━━━━━━━━ 76s 197ms/step - accuracy: 0.0753 - loss: 8.6468 - val_accuracy: 0.0844 - val_loss: 8.2606
Epoch 3/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.0851 - loss: 8.1240Epoch 3 - Perplexity: 3353.31
283/283 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.0850 - loss: 8.1240 - val_accuracy: 0.0844 - val_loss: 8.1177
Epoch 4/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.0791 - loss: 7.9792Epoch 4 - Perplexity: 3200.26
283/283 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.0791 - loss: 7.9794 - val_accuracy: 0.0844 - val_loss: 8.0710
Epoch 5/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/s

**OBSERVACIÓN:**

**Por lo que se puede observar de los resultados, el modelo no está entrenando de manera efectiva. Aunque la pérdida (loss) está disminuyendo lentamente, la precisión (accuracy) se mantiene muy baja (alrededor de 8% en todos los casos, tanto para el conjunto de entrenamiento como para el conjunto de validación). Además, la perplejidad sigue siendo extremadamente alta, lo que indica que el modelo tiene dificultades para predecir correctamente las secuencias.**

**Mejoras Aplicadas:**

. Preprocesamiento: Se ha mejorado la limpieza de texto eliminando caracteres no alfanuméricos y números.

. Vocabulario: El vocabulario se ha limitado a las 10,000 palabras más frecuentes (num_words=10000), lo que ayuda a reducir el espacio de búsqueda y a hacer el aprendizaje más manejable.
. Longitud de secuencias: La longitud máxima de las secuencias se ha reducido a 50 palabras (max_sequence_length=50), lo que reduce la complejidad del modelo.

. Arquitectura del modelo:

Se han añadido dos capas LSTM. La primera capa tiene return_sequences=True para permitir la entrada de una segunda capa LSTM.
Cada capa tiene dropout y recurrent dropout para ayudar a evitar el sobreajuste.

. Cálculo de Perplejidad: Se ha agregado un callback personalizado para calcular la perplejidad al final de cada época.

. Evaluación: Después de entrenar, se realiza una evaluación en el conjunto de validación y se imprimen las métricas de pérdida y precisión.

In [19]:
# Función para preprocesar el texto
def preprocess_text(text):
    text = text.lower()  # Convertimos a minúsculas
    text = re.sub(r'\W+', ' ', text)  # Eliminamos caracteres no alfanuméricos
    text = re.sub(r'\d+', '', text)  # Eliminamos números
    return text

texts = [preprocess_text(text) for text in newsgroups.data]

# Etiquetas
labels = newsgroups.target

# Configuración del Tokenizer
vocab_size = 10000  # Reducimos el vocabulario a las 10,000 palabras más frecuentes
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)

# Convertimos textos a secuencias
sequences = tokenizer.texts_to_sequences(texts)

# Definimos la longitud máxima de secuencias
max_sequence_length = 50  # Reducimos la longitud de las secuencias
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Dividimos el dataset en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.2, random_state=42)

# Construcción del modelo LSTM mejorado
model = Sequential()

# Capa de embedding
embedding_dim = 128
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))

# Primera capa LSTM con return_sequences para más profundidad
hidden_units = 128
model.add(LSTM(hidden_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

# Segunda capa LSTM
model.add(LSTM(hidden_units, dropout=0.2, recurrent_dropout=0.2))

# Capa densa de salida
model.add(Dense(20, activation='softmax'))  # 20 clases en el dataset 20 Newsgroups

# Compilación del modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callback personalizado para calcular Perplejidad
class PerplexityCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        perplexity = tf.exp(logs["loss"]).numpy()  # Calcular la perplejidad
        print(f"Epoch {epoch + 1} - Perplexity: {perplexity:.2f}")

# Entrenamiento del modelo
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=[PerplexityCallback()]
)

# Evaluación del modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

Epoch 1/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.1010 - loss: 2.8242Epoch 1 - Perplexity: 13.54
283/283 ━━━━━━━━━━━━━━━━━━━━ 53s 171ms/step - accuracy: 0.1012 - loss: 2.8234 - val_accuracy: 0.2669 - val_loss: 2.2357
Epoch 2/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.3066 - loss: 2.0284Epoch 2 - Perplexity: 6.84
283/283 ━━━━━━━━━━━━━━━━━━━━ 49s 173ms/step - accuracy: 0.3068 - loss: 2.0280 - val_accuracy: 0.3805 - val_loss: 1.8772
Epoch 3/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.5052 - loss: 1.4621Epoch 3 - Perplexity: 4.23
283/283 ━━━━━━━━━━━━━━━━━━━━ 79s 164ms/step - accuracy: 0.5053 - loss: 1.4620 - val_accuracy: 0.4839 - val_loss: 1.6345
Epoch 4/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.6321 - loss: 1.1088Epoch 4 - Perplexity: 3.03
283/283 ━━━━━━━━━━━━━━━━━━━━ 83s 166ms/step - accuracy: 0.6321 - loss: 1.1088 - val_accuracy: 0.5245 - val_loss: 1.5962
Epoch 5/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accurac

**CONCLUSIÓN**

**El modelo muestra una mejora significativa en precisión, alcanzando el 91.72% en entrenamiento y 61.15% en validación. Las métricas de pérdida han mostrado una tendencia a la baja, indicando que el modelo está aprendiendo de manera efectiva.
La perplejidad también ha mejorado a lo largo de las épocas, disminuyendo de 13.54 a 1.30, lo que sugiere una mejor comprensión del modelo sobre los datos.
Sin embargo, se observan leves signos de sobreajuste en las últimas épocas, ya que la pérdida de validación no disminuye de manera similar.
En general, el modelo presenta un buen potencial para clasificar correctamente las categorias del dataset.**

**Generación de Texto - Greedy Search**

In [31]:
def generate_text_sample(model, tokenizer, seed_text, num_words):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_word_index = np.random.choice(len(predicted_probs), p=predicted_probs/np.sum(predicted_probs))  # Muestreo
        predicted_word = tokenizer.index_word.get(predicted_word_index, '')
        if not predicted_word:
            break
        seed_text += ' ' + predicted_word
    return seed_text

seed_text = "the impact of artificial intelligence on society"
generated_text = generate_text_sample(model, tokenizer, seed_text, num_words=50)
print(f'Generated text (Sample): {generated_text}')

Generated text (Sample): the impact of artificial intelligence on society on you is


**Generación de texto - Beam Search**

In [32]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature  # Evitar log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def beam_search_predict(model, tokenizer, seed_text, beam_width=3, max_sequence_length=50, temperature=1.0):
    sequences = [[seed_text, 0.0]]  # Secuencias iniciales y sus puntajes
    for _ in range(max_sequence_length):
        all_candidates = []
        for seq, score in sequences:
            token_list = tokenizer.texts_to_sequences([seq])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
            predicted_probs = model.predict(token_list, verbose=0)[0]
            top_k = np.argsort(predicted_probs)[-beam_width:]  # Obtener las top_k palabras

            for word_index in top_k:
                # Usar el método de muestreo
                sampled_word_index = sample(predicted_probs, temperature)
                predicted_word = tokenizer.index_word.get(sampled_word_index, '')

                if predicted_word:  # Asegúrate de que no sea una palabra vacía
                    candidate = seq + ' ' + predicted_word
                    candidate_score = score - np.log(predicted_probs[sampled_word_index] + 1e-10)  # Agregar el puntaje
                    all_candidates.append([candidate, candidate_score])

        sequences = sorted(all_candidates, key=lambda x: x[1])[:beam_width]  # Mantener las mejores

    return sequences[0][0]

# Generamos texto usando la función ajustada
generated_text_beam = beam_search_predict(model, tokenizer, seed_text, beam_width=10, temperature=0.5)
print(f'Generated text (Beam Search): {generated_text_beam}')

Generated text (Beam Search): the impact of artificial intelligence on society on on on on on on on on this this this this this this you you you you you you you you you you you it it it it it it it i i i i to to to to to to to to to ax ax to to to


**Generación de texto con Temperatura - Busqueda Estocástica**

In [34]:
def stochastic_search_predict(model, tokenizer, seed_text, num_words, temperature=1.0, max_sequence_length=50):
    generated_text = seed_text

    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')

        # Predicciones de probabilidad
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Aplicamos la temperatura
        exp_probs = np.exp(predicted_probs / temperature)
        prob_distribution = exp_probs / np.sum(exp_probs)  # Normalización

        # Muestreo de la distribución de probabilidad
        predicted_word_index = np.random.choice(range(len(prob_distribution)), p=prob_distribution)
        predicted_word = tokenizer.index_word.get(predicted_word_index, '')

        if not predicted_word:
            break

        generated_text += ' ' + predicted_word

    return generated_text

# Parámetros
seed_text = "the impact of artificial intelligence on society"
generated_text_stochastic = stochastic_search_predict(model, tokenizer, seed_text, num_words=50, temperature=0.7)
print(f'Generated text (Stochastic Search): {generated_text_stochastic}')

Generated text (Stochastic Search): the impact of artificial intelligence on society edu it ax on to ax from is t edu is on on you and this in is ax it you you you is from ax of this a


**CONCLUSIÓN**

**En la generación de texto, se probaron tres estrategias: Greedy Search, Beam Search y Búsqueda Estocástica. La Greedy Search generó resultados limitados. la Beam Search mostró una ligera mejora al considerar múltiples secuencias, aunque aún presentaba ciertas redundancias. Finalmente, la Búsqueda Estocástica ofreció los resultados más variados y coherentes, al introducir aleatoriedad mediante el muestreo de palabras, lo que permitió una generación de texto más rica y creativa. En conclusión, la Búsqueda Estocástica fue la más efectiva para lograr diversidad y fluidez en el texto generado.**